In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from tabulate import tabulate
from gensim.models import Word2Vec
from tabulate import tabulate
# Descargar recursos necesarios de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

C:\Users\User\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# Cargar y preprocesar los datos
datos = pd.read_csv('datos4v5.csv', encoding='utf-8', delimiter=';')
etiquetas = datos['autor']
textos = datos['texto']

In [3]:
# Preprocesamiento de texto
def preprocesar_texto(texto):
    stop_words = set(stopwords.words('spanish'))
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(texto.lower())
    tokens = [token for token in tokens if token.isalpha()]
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return " ".join(tokens)

In [4]:
# Crear el vectorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000)

In [5]:
# Preprocesar y vectorizar los textos
textos_limpio = [preprocesar_texto(texto) for texto in textos]
X_tfidf = tfidf_vectorizer.fit_transform(textos_limpio)

In [6]:
# Modelo Word Embedding
model_w2v = Word2Vec(sentences=[word_tokenize(texto) for texto in textos_limpio], vector_size=100, window=5, min_count=1, sg=0)

In [7]:
# Función para obtener vector promedio de Word Embedding
def obtener_vector_promedio(texto, model):
    vectors = [model.wv[word] for word in word_tokenize(texto) if word in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    return np.zeros(model.vector_size)  # Asegura que el vector tenga el mismo tamaño que el modelo


In [8]:
# Crear los vectores de Word Embedding
X_w2v = np.array([obtener_vector_promedio(texto, model_w2v) for texto in textos_limpio])

# Asegurarse de que X_w2v tenga las dimensiones adecuadas (2D)
X_w2v = X_w2v.reshape(X_w2v.shape[0], -1)

In [9]:
# Combinar características de TF-IDF y Word Embedding
X_combined = np.hstack((X_tfidf.toarray(), X_w2v))

In [10]:
# Escalar características al rango [0, 1]
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_combined)

# Codificar las etiquetas
label_encoder = LabelEncoder()
etiquetas_encoded = label_encoder.fit_transform(etiquetas)


In [11]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_combined, etiquetas_encoded, test_size=0.3, random_state=42)

In [12]:
# Entrenar y evaluar modelos
modelos = {
    'SVC Lineal': SVC(kernel='linear', probability=True),
    'SVC RBF': SVC(kernel='rbf', probability=True),
    'Random Forest': RandomForestClassifier(n_estimators=100),
    'Regresion Logistica': LogisticRegression()
   
}

resultados = {}

for nombre, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    conf_matrix = confusion_matrix(y_test, y_pred)
    resultados[nombre] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'Confusion Matrix': conf_matrix
    }

C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning

In [13]:
# Mostrar resultados en una tabla
resultados_df = pd.DataFrame(resultados).T
print(tabulate(resultados_df, headers='keys', tablefmt='grid'))

+---------------------+------------+-------------+----------+-------------------------+
|                     |   Accuracy |   Precision |   Recall | Confusion Matrix        |
+=====================+============+=============+==========+=========================+
| SVC Lineal          |       0.75 |    0.808333 |     0.75 | [[2 0 0 0 0 0 0 0 0]    |
|                     |            |             |          |  [0 3 0 0 0 0 0 0 0]    |
|                     |            |             |          |  [1 0 1 0 0 0 0 0 0]    |
|                     |            |             |          |  [0 0 0 0 0 0 0 1 0]    |
|                     |            |             |          |  [0 0 0 0 2 0 0 0 0]    |
|                     |            |             |          |  [0 1 0 0 0 2 0 0 0]    |
|                     |            |             |          |  [0 2 0 0 0 0 1 0 0]    |
|                     |            |             |          |  [0 0 0 0 0 0 0 2 0]    |
|                     |         

In [65]:
# Cargar nuevo artículo
#Articulo nuevo autor1 - no esta en base de datos
#nuevo_articulo = "La resolución de problemas es un tema que en los últimos años se lo aborda con gran interés y preocupación por parte de las instituciones educativa; surge como resultado de considerar el aprendizaje como una construcción social que encierra cálculos, pruebas y contradicciones basado en un proceso creativo y generativo (Coronel & Curotto, 2008). Los profesionales están continuamente expuestos a enfrentar situaciones problemáticas y estresantes que demanda la sociedad actual, él debe ser capaz de procesar información, generar conocimiento útil y tomar decisiones, porque necesita mantener una comunicación asertiva e interdependencia en sus acciones para implicarse con intensidad, energía y responsabilidad en las tareas encomendadas (Arteaga, 2009; Carles, 2001; León, Castaño, Iglesias, & Marugán, 2014; Rigo & Donolo, 2014). Por tanto, es necesario que su formación esté dotada de conocimientos, habilidades y capacidades que lo hagan más competitivo social y profesionalmente. Desempeñarse en el campo laboral y alcanzar metas, implica afrontar y resolver obstáculos que aparecen en el camino. Por lo tanto, debe entrenarse al educando desde las aulas para que aprenda a controlar su aprendizaje y rendimiento que permita desarrollar una madurez actitudinal en su ocupación (Torrano Montalvo & González Torres, 2004). Hay estudios que revelan que su formación debe estar acorde con las necesidades del espacio laboral y que deben saber afrontar y resolver problemas con decisión, eficacia, entrega y compromiso para alcanzar objetivos con éxito (Bados & García, 2014; Torres, 2007; Torres, Badillo, Valentin, & Ramírez, 2014). Ante estos hechos, qué están haciendo las universidades para desarrollar estas competencias en el estudiantado. Lograr estos cambios, requerirá de reformas en el currículum de estudio, donde no solamente se prioricen contenidos disciplinares sino también interdisciplinarios, que tengan un enfoque en competencias con rasgos distintivos, que permitan un desempeño satisfactorio en situaciones reales de trabajo (Moreno T. , 2010). Estos cambios formativos han sido fomentados en los Espacios Europeos de Educación Superior (EEES) para el desarrollo de competencia con un mayor protagonismo del estudiantado en su aprendizaje (González, 2011), en esta misma línea América Latina, ha seguido los pasos en este cambio de estructura formativa en la que incluyen destrezas, conocimientos y contenidos (Proyecto Tuning América Latina, 2011-2013), y Ecuador participa en ella con 12 universidades entre públicas y privadas para fomentar la formación en sus aulas. En esta misma línea, Biggs 2006 citado por Cardeñoso (2012) defiende que el estudiantado universitario, debe implicarse en tareas de aprendizaje de orden superior, y así aprenda a reflexionar críticamente, argumentar y poner en práctica la resolución de casos, problemas o proyectos, esto guarda relación con González y Wagenaar (2006) al mencionar que la competencia de resolución de problemas es una de las competencias más requeridas por los empleadores y graduados en el campo laboral y personal. Con estos antecedentes, se pretende conocer qué competencias de resolución de problemas utiliza el estudiantado universitario, de manera particular en las universidades presenciales de la provincia de Esmeraldas, en las carreras de Ingeniería Mecánica de la Universidad Luis Vargas Torres LVT e Ingeniería en Sistemas Informáticos de la Pontificia Universidad Católica del Ecuador Sede Esmeraldas PUCESE, para analizar los estilos de resolución de problemas que utiliza el estudiantado universitario de acuerdo al género y nivel que cursa. La educación superior en los últimos años ha sufrido procesos de cambios relacionados con la calidad educativa y formativa del profesorado, quien tiene un radio de acción más amplio, porque no solo le corresponde la enseñanza, sino, además, la investigación y otros aspectos que empoderan su rol como académico. Estos cambios en busca de la calidad que está experimentando la universidad, conlleva a modificaciones en las funciones roles y tareas que desempeña el docente, que le exige desarrollar nuevas competencias para desempeñar de manera eficaz sus labores profesionales (Mas, 2011). Ante ello, es necesario que se incorporen mecanismos de formación y evaluación que permita mejorar la calidad docente como parte fundamental del desarrollo profesional y de la identidad docente (Caballero & Bolívar, 2015). La identidad profesional es el conjunto de atributos que caracterizan a un grupo de personas que poseen cierto grado de educación para ejercer un trabajo integral con calidad y compromiso, permitiéndole reconocerse y ser reconocido como un integrante de un gremio, repercutiendo en la vida personal y social del individuo (Caballero 2009; Muñoz & Arvayo 2015). Pues bien, en los académicos, este auto-reconocimiento es conocido como identidad docente; representación que podría mantenerse en el tiempo, mediante contenidos definido a través de roles, creencias y sentimientos que se enlazan al ejercer la profesión en determinados ambientes donde desarrolla su práctica docente (Gómez & Niño 2015). Estos roles del docente en el contexto universitario van a determinar en gran medida las opiniones que desarrollen sobre su profesión y la construcción de una identidad académica (Caballero & Bolívar, 2015), considerada como una identidad específica, donde su resultado es producto de un proceso de socialización entre profesionales que pertenecen a un grupo y se relaciona con el trabajo que se realiza en un ambiente particular (Alonso, Lobato, & Arandia, 2015). Por tanto, es una construcción social que se logra desde que se nace y se va formando, cambiando y modificando, producto de los intercambios que resulta entre actores sociales que participan de un mismo grupo (Salcedo, 2015). Esta transformación involucra los modelos de desempeño de los roles de docentes e investigadores, la forma de comprender la constitución de sus identidades académicas (Larrinaga, 2016), sus pensamientos sobre la enseñanza aprendizaje y la investigación. Es esta doble función de formar e investigar la que da relevancia a la docencia, manteniéndose a la vanguardia de los procesos construidos entre docentes, estudiantes e investigadores (Hernández, 2009). Construir la identidad en los académicos es muy difícil, más aún cuando intervienen profesionales de distintas disciplinas, químicos, médicos, investigador o autores de artículos científicos en revistas de alto impacto (Muñoz & Arvayo 2015). De esta manera, la identidad del profesorado y por ende la del investigador, sería el resultado de varias interacciones entre la experiencia personal y el ambiente social-cultural e institucional dentro del cual se desenvuelve habitualmente el profesional (Aguayo, Castelló & Monereo, 2014). Este trabajo de investigación nace producto de un estudio a nivel de Posgrado realizado en el año 2015 en la Pontificia Universidad Católica del Ecuador Esmeraldas (PUCE-E), concretamente en la Escuela de Contabilidad y Auditoría, donde los resultados revelaron que la función investigadora del profesorado ha sido poco desarrollada generado debilidades en su formación como futuros investigadores (Achilie , 2017); hecho que es subrayado en el estudio realizado por Pazmiño (2016). Por otro lado, los nuevos retos que ha asumido el docente, la investigación, ha traído como efecto su vinculación en diferentes disciplinas o áreas en la que posiblemente no se ha identificado; o ha recibido poca preparación durante su formación (Aguayo, Castelló, & Monereo, 2014). Estos elementos podrían haber generado, nerviosismo, inseguridad en su identidad al desarrollar su tarea docente, especialmente en lo concerniente a sus actividades investigativas. El estudiar la identidad del académico es importante, por cuanto las instituciones de Educación Superior están viviendo procesos de cambio que involucra al docente a asumir nuevos roles, identidades, desarrollar competencias (Cruz, 2015), direccionada al mejoramiento de su perfil profesional; permitiéndole participar en redes de investigación, asesorías, difundir su producción científica y vincularse hasta en procesos de gestión en la universidad (Trujillo, 2015). Además, este estudio daría pautas a la PUCE-E para que busque alianzas estratégicas con otras universidades a nivel local, nacional e internacional, con la finalidad de establecer nuevos planes de formación y desarrollo profesional, que conlleve a seguir fortaleciendo la actividad investigadora; más aún cuando el Plan Nacional del Buen Vivir (2013) en su cuarto objetivo establece los lineamientos estratégicos encaminados al mejoramiento de la educación. Asimismo, la LOES (2010) en su art 13 literal d, señala la necesidad de fortalecer el desarrollo de la docencia y la investigación en todos los niveles y modalidades del sistema superior, lo que es subrayado en la misma Ley en su art 6 literal a. Además, podría ser beneficioso asegurar la incorporación de profesionales que motiven el interés entre otros aspectos, por la investigación, indicador de productividad para la universidad, que a la vez posibilita dar pautas para tener un acercamiento con los docentes que permita resolver la interrogante ¿Cómo se identifica el profesorado en su rol investigador? Basados en estos antecedentes y en cuanto pueda aportar a la institución, específicamente a la Escuela de Contabilidad y Auditoría de la PUCESE, se planteó como objetivo, describir la identidad del docente en su rol de investigador. Los desastres naturales han estado ligados en la historia de la humanidad por su poder destructivo y efectos negativos perdurables en la sociedad  (Espinoza, 2008).  Los ecuatorianos hace pocos años experimentaron estos eventos con el sismo de abril 16 de 2016.  Las provincias que sufrieron mayores afectaciones fueron Manabí y Esmeraldas;  en  el caso de esta última, se evidenció que los mayores daños sociales, económicos y materiales fueron en la Isla del cantón Muisne. Las familias de la isla tuvieron que dejar sus viviendas, negocios y todo su diario vivir para albergarse en zonas de ayuda gubernamental. Estos eventos dejan daños en las viviendas que no siempre cuentan con el apoyo ni el financiamiento del estado para sus reparaciones, en muchos de los casos deben ser financiados con recursos propios de los afectados (Maldonado & Albio, 2013). En cuanto a las actividades del comercio en Muisne, las tiendas de abarrotes son las que dan mayor generación económica.  Al mismo tiempo, fueron las más afectadas por cuanto se redujeron sus ventas debido a que las familias estaban albergadas en centros de ayuda del estado. Además de ello, el mal estado de las carreteras que dificultaban el acceso de los proveedores con las provisiones.     Aunado a esto, un estudio sobre secuelas del sismo de 2016 desarrollado por Ruiz (2017),  muestra  el cambio del comportamiento en compras un año después del terremoto en una empresa dedicada al comercio menor.  Se pudo observar que los clientes de Manabí redujeron sus compras en un 29% en relación con el año anterior; y que, además el 33% de los clientes que compraron en el periodo antes del terremoto no volvieron a comprar después del mismo.  Es por ello necesario que se dé apoyo a las pequeñas y medianas empresas que en muchos casos no pueden retomar sus actividades después de estas catástrofes  (Dresdner & Sehnbruch, 2010; Mella, 2012).    Estos desastres,  inmovilizan temporalmente la economía, puesto que la destrucción en infraestructura y equipamiento del sector productivo, imposibilitan su funcionamiento  (Rasse & Letelier,  2012; Razeto,  2013).  Hay países como es el caso de Chile que han experimentado en repitas ocasiones estos eventos, uno a citar es el del 27 de febrero de 2010 en el que desarrollaron el Plan de Reconstrucción Terremoto y Maremoto para apoyar especialmente a Pymes y pescadores artesanales con la creación de incentivos tributarios y financieros para la reconstrucción de infraestructura y equipamiento de las zonas más afectadas por el sismo  (Gobierno de Chile,  2010). Además de ello, se puede citar el Programa de Desarrollo Solidario (PDS) del Banco del Desarrollo que se lleva a cabo en Chile y brinda apoyo financiero y formativo a más de 10.000 mujeres de bajos recursos y aproximadamente el 60% de ellas los destinan al mejoramiento de sus viviendas (Rojas, 2017), todo esto con el fin de poner en práctica alternativas de recuperación de las zonas afectadas.    En  cuanto al  gobierno ecuatoriano  luego de  sismo,  planteó  políticas,  programas  y estrategias de reactivación económica de la población  afectada,  a  través  de  la  Ley Orgánica  de  Solidaridad  y  de Corresponsabilidad  Ciudadana  para  la Reconstrucción  y  Reactivación  (LOSCCRR) de  las  zonas  afectadas,  a  través  de  la contribución  solidaria  sobre  la remuneración,  el  patrimonio,  bienes inmuebles,  utilidades,  exoneración  y/o devolución  del  anticipo  de  impuesto  a  la renta,  incremento  de  tarifa  IVA  al  14%, excepto a las zonas afectas, y la exoneración del pago de cuotas de Régimen Simplificado RISE (LOSCCRR, 2016).    De acuerdo con informes del Plan Reconstruyo Ecuador,  las  mayores afectaciones se concentraron en las empresas grandes del sector formal en un 43,40% y las empresas pequeñas en 31,10%  (Comite de Reconstrucción y Reactivación Productiva, 2016). Además, las gestiones de reactivación se dieron también en el sector financiero como es el caso del Ban Ecuador, que realizó aproximadamente 2.251 operaciones de crédito en beneficio de los pobladores de Muisne a partir del 9 de mayo de 2016, con un monto superior a USD 6.5 millones destinados a las distintas actividades productivas de la zona  (Ban Ecuador, 2017).     Luego de aproximadamente un año y medio de este evento natural,  se  ha considerado pertinente analizar los beneficios a los que accedieron los comerciantes de víveres del cantón Muisne, en virtud de los incentivos promovidos por la LOSCCRR. Desde inicios del siglo XXI, las Instituciones de Educación Superior, así como los docentes universitarios,  están  viviendo un proceso de transformación en sus diferentes dimensiones, debido a varios cambios a nivel social, cultural, educativos, tecnológicos económicos y políticos. En este marco, el docente universitario ecuatoriano ha encontrado nuevos y variados retos y desafíos, entre ellos la investigación. Esta actividad ha sido un elemento importante debido a que conduce a la generación de nuevos conocimientos, como lo concibe la Organización de las Naciones Unidas para la Educación, la Ciencia y la Cultura (UNESCO, 2005) y en los últimos años, ha asumido un rol protagónico (Segura, 2008). Además, esta misión ha generado debates importantes, en el caso de la academia ecuatoriana, debido  a las nuevas obligaciones en la actividad del docente (Cortés, 2006, p. 9).Así, dentro de las actividades de investigación que han sido consideradas de envergadura para el fortalecimiento y cambio de las universidades ecuatorianas1, la producción científica ha sido el factor que formado parte de los debates en los ámbitos políticos, académicos, culturales, sociales, entre otros, de forma significativa,  esto teniendo  en cuenta que en  el contexto global latino  e iberoamericano la  producción  científica ecuatoriana  es baja  y  poco visible  internacionalmente (Bruque, 2013).  La investigación científica, como uno de los motores de la academia ecuatoriana, de hecho, ha formado parte de las políticas transformadoras del país. Es así que, el Plan Nacional del Buen Vivir 2013-2017, ha definido entre sus cambios más importantes: impulsar la transformación de la matriz productiva, articulando la investigación científica, tecnológica y la unión de la Educación Superior con el sector productivo, para conseguir una mejora permanente de la productividad y la competitividad de manera integral. En este contexto, varias universidades del país han comenzado procesos de auditorías (internas y externas) para conocer sus capacidades, fortalezas y debilidades en los distintos aspectos de la academia: docencia, gestión e investigación. Una de estas universidades ha sido la Pontificia Universidad Católica del Ecuador Sede Esmeraldas (PUCESE), que al ser evaluada por el anterior Consejo  Nacional  de  Evaluación  y  Acreditación  de  la  Educación  Superior  (CONEA,  2010), mostró  un  escenario deslucido  en  la producción científica,  con  un 12,07%  en  relación a  los otros indicadores (academia, gestión  y extensión), y con este porcentaje la Sede Esmeraldas no alcanzó los índices necesarios  en investigación de acuerdo al número de sus docentes. Sobre esta problemática,  varios han sido los aportes que se han enfocado en la producción investigativa. Así, por ejemplo, existen investigaciones que han analizado la opinión del docente universitario en contabilidad en relación a su producción científica y las causas que pueden explicar su diferencia respecto a otras disciplinas (Escobar et al., 2014). Otros estudios se han enfocado en conocer el estado de la investigación de las ciencias económicas, administrativas y contables en relación a otras áreas de estudio (Flores, 2011), así como también, las dificultades de esta área en la producción investigativa (Briceño et al., 2007). Asimismo, existen investigaciones que han valorado el papel del departamento de postgrado en la formación de investigadores contables (Viloria, Casal, Peña, 2010).Empero,  en  el  Ecuador,  los  estudios  sobre  esta  problemática  han  sido  escasos  y  se  han enfocado sobre todo  en  analizar de manera  bibliométrico-comparativa  la  producción científica del país, exponiendo que en el contexto global latino e iberoamericano la producción científica ecuatoriana es baja y poco visible internacionalmente (Bruque, 2013). Así,  esta  investigación tiene como objetivo analizar los elementos que contribuyen o limitan la producción investigativa de la PUCESE, en particular, de los docentes de la Escuela de Contabilidad y Auditoría.El Plan de  gobierno  ecuatoriano realiza  reformas  en  la  administración  de tributos para financiar la educación, la salud y  la  seguridad  social  (Paz  &  Miño,  2015). Estos cambios traen  consigo  el  seguir fomentando la cultura tributaria en todos los niveles. De acuerdo al Ministerio de Finanzas (2015) los  impuestos  representan  el 63.76%  de  los  ingresos  permanentes, siendo  los  de  mayor  recaudación  el impuesto a la renta, seguido del impuesto al valor  agregado  (IVA).  Este  último  modelo fiscal tiene dos tipos de contribuyentes, los clasificados  en  el  Registro  Único  de Contribuyentes (RUC) y los  que conforman el  sistema  de  Régimen  Impositivo Simplificado (RISE), en el  que  se cancelan cuotas  mensuales  que  reemplazan  la obligación de realizar declaraciones de IVA e impuesto a la renta. (SRI, 2010)  La  presencia  del  RISE  desde  el  año 2008  ha  permitido  que  algunos  sectores como  el  comercio  y  servicio  puedan contribuir  a  la  formalización  de  los pequeños  negocios  y  a  su  crecimiento, mediante  el  alivio  de  la  carga  burocrática que  enfrentan,  y  potenciar  sus  beneficios” (Granda  &  Zambrano  ,  2012,  p.  13).  Esa experiencia ya  la  han  vivido  otros  países, como  Argentina  con  el  Monotributo,  en  el que según Salim y D’Angela (2006), tuvo éxito  porque  permitió  el  incremento  en  la recaudación, al ser alícuotas fijas; pero a la vez  mencionan  que  se  ha  perdido participación  en  relación  a  la  recaudación total. Otro caso  a citar es  el de Colombia, donde Sabaíni y Morán (2016) insisten en la necesidad de buscar alternativas en materia de  IVA,  que  regulen  su  recaudación  a  los sectores más vulnerables y que se lo podría relacionar  con  lo  que  indica  Rodríguez (2014)  sobre  la  necesidad  de  que  estos cambios  no  deben  caer  en  estimaciones presuntivas,  que  pongan  en  riesgo  la equidad  tributaria  entre  contribuyentes; además,  Granda  y  Zambrano  (2012) comentan que existe mucha desinformación sobre el RISE en relación a los beneficios y obligaciones que la ley determina.  Al  ser  el  pago  de  tributos  una obligación  de  todo  contribuyente,  fue necesario  buscar  un  sector  que  concentre comerciantes  con  diferentes  modelos fiscales,  como  es  el  caso  del  mercado municipal  y  se  investiguen  qué  problemas tributarios pueden tener, ya que este sector agremia  a  107  comerciantes  que contribuyen  con  el  dinamismo  de  la economía de la ciudad y que a la vez dieron la  pauta  para  que  esta  investigación  se plantee  como  objetivo  el  analizar  la situación  tributaria  de  un  grupo  de comerciantes de legumbres y productos de consumo masivo  del  Mercado  Municipal de la ciudad de Esmeraldas."

#Articulo en base de datos autor2 - si esta en la base de datos
#nuevo_articulo="La diabetes mellitus es considerada una epidemia a nivel mundial, cobrando cada vez más casos en la población y convirtiéndose en una de las enfermedades crónicas no transmisibles con mayor mortalidad y mayor coste a los ministerios de salud en todo el mundo [1].Según la Federación  Internacional de Diabetes [2], en el año 2015 hubo 415 millones de adultos entre los 20 y 79 años con diagnóstico de diabetes a nivel mundial,  incluyendo  193  millones  que aún  no  estuvieron diagnosticados. Además, se considera que  existen 318 millones de adultos con alteración en la tolerancia a la glucosa, los que presentan un alto riesgo de desarrollar diabetes en los próximos años. De esta manera se es-tima que para el año 2040 existirán en el mundo 642 millones de personas viviendo con esta enfermedad [3], [4]. A nivel mundial se le atribuye a la diabetes mellitus aproximadamente 4.6 millones de defunciones al año [3] . Esta enfermedad se encuentra dentro de las 10 pri-meras causas de discapacidad en el mundo [4], disminuyendo la productividad en los sectores industriales, productivos de los países y afectando de forma considerable el desarrollo humano de los pueblos. La diabetes mellitus consiste en una alteración metabólica, que se caracteriza por la presencia continua de glucosa en el torrente sanguíneo, producido por una irregularidad en la producción de insulina o una resistencia por la acción de la misma, y en ocasiones a una combinación de varios factores. La diabetes está relacionada con algunas complicaciones de la salud, entre las que se mencionan la pérdida de la visión, alteraciones de los riñones, irregularidades en los vasos sanguíneos, enfermedades coronarias, problemas cerebrales e irrigación intestinal, afectándose el sistema nervioso periférico y autónomo. Considerando las implicaciones de la diabetes mellitus, se ha realizado un estudio para la evaluación de factores de riesgos en las personas, con el propósito de realizar un diagnóstico precoz, un control acorde a las complicaciones individuales de las personas y promover la prevención y los cuidados necesarios para evitar el problema de salud. En el año 2014 el Instituto Nacional de Estadística y Censos del Ecuador [5], reportó como segunda causa de mortalidad general a la diabetes mellitus, situándose además como la primera causa de mortalidad en la población femenina y la tercera en la población masculina. Los pacientes con diabetes mellitus desarrollan complicaciones a largo plazo, no siendo la intensidad y duración de la hiperglucemia los únicos factores determinantes para la aparición de dichas complicaciones. Algunos otros factores desencadenan estas complicaciones crónicas, trayendo consigo un aumento en su morbilidad y su mortalidad, además de la invalidez en este grupo poblacional. El embarazo es un proceso natural en el  que  la  mujer  sufre  muchos  cambios fisiológicos, quedando expuesta a múltiples complicaciones  que  alteran  su  evolución normal y se traducen en elevadas tasas de morbimortalidad  materna  y  perinatal. (García, 1996).  La  enfermedad  hipertensiva  del embarazo es una de las problemáticas más frecuentes en este grupo de pacientes. Bajo este término se engloba una gran variedad de  procesos  que  tienen  en  común  la existencia de hipertensión arterial durante la gestación (Amaran,2009).  La  frecuencia  de la  enfermedad hipertensiva del embarazo varía entre el 5% y 10% en los países desarrollados, pero esta cifra podría elevarse hasta alcanzar el 18% en los países en desarrollo; se estima que es de  un  caso  por  cada  100  mujeres.  En algunos  países  en  vías  de  desarrollo,  su incidencia representa entre el 40% y el 80% de  las  muertes  maternas,  siendo responsable  de  entre  63,000  a  200,000 muertes anuales a nivel mundial. Además, la mortalidad perinatal se quintuplica en las mujeres  con  enfermedad  hipertensiva  del embarazo,  con  frecuencia  debido  a  la restricción del  crecimiento  intrauterino  y  a los partos pretérminos (Sánchez, 2004)  En América Latina, una cuarta parte de las muertes maternas han sido asociadas con  las  complicaciones  derivadas  de  los trastornos  hipertensivos  gestacionales;  la preeclampsia y la eclampsia destacan como principales  causas  de  morbilidad  y mortalidad  materna  y  perinatal.  Estas muertes,  en  su  mayoría,  son  evitables mediante la prestación de atención oportuna y eficaz a las mujeres que presentan una de estas  complicaciones  (Guía  de  Prácticas Clínicas, 2013). La  enfermedad  hipertensiva  del embarazo  es  la  primera  causa  de  muerte materna en el Ecuador; entre tres y cuatro de  cada  cien  mujeres  fallecen  por  este motivo.  Por  su  incidencia  decenas  de científicos en todo el mundo se han dedicado a investigar cuáles son sus causas (Guía de Prácticas Clínicas, 2013). Para pacientes sin factores de riesgo su incidencia mundial se calcula entre 3% a 8  %,  y  en  mujeres  con  factores  de  riesgo entre 15% a 20 %. Por el alto por ciento de incidencia de esta enfermedad en pacientes con  factores  de  riesgo,  es  de  vital importancia  la  prevención  de  los  mismos desde la perspectiva de la atención primaria de salud (Steegers, 2010).  Por  todo  lo  antes  expuesto,  nos dimos  a  la  tarea  de  realizar  esta investigación  para  contribuir  a  un  mayor conocimiento de los factores de riesgo de la enfermedad hipertensiva del embarazo en el barrio  Santa  Cruz,  de  la  ciudad  de Esmeraldas, a partir de la determinación de los factores de riesgo de dicha enfermedad presentes en la comunidad."

#Articulo en base de datos autor3 - no esta en base de datos
#nuevo_articulo="Los esfuerzos para mejorar  la educación en Ecuador desde el cambio de gobierno en 2007 y al amparo de la Constitución de 2008 (que  reconoce  en  los  artículos 26  y  28  el derecho  y  obligatoriedad  a  la  educación gratuita  y  de  calidad  desde  inicial  hasta bachillerato  o  equivalente),  se  han manifestado  en  diversos  documentos  de planificación  y  diseño  de  políticas educativas. Además, se contempla la necesidad  de  impulsar  una  ética  de  la inclusión  con  acciones  afirmativas  y  una cultura  escolar  inclusiva  y  tendente  a eliminar  toda  forma  de  discriminación.  En ese marco de potenciación de la educación inclusiva,  el  Acuerdo  Ministerial  0295-13 (2013) del Ministerio de Educación formula en  el  artículo  3  la  inclinación  hacia  la promoción  e  inclusión  de  quienes  puedan acceder  a  instituciones  de  educación ordinaria desde la educación especializada. Dichos  referentes  son  coincidentes  con  la concepción de  la  inclusión de  Bustamante, Terán e Hinojosa, quienes la definen como aquella situación en la que: el  niño/a  o  adolescente  se  incluye oportunamente,  en  la  edad  adecuada, en  la  educación  inicial,  básica  y bachillerato (o su equivalente): accede, progresa y es bien tratado/a dentro del sistema educativo; avanza sin rezago y concluye su educación. El buen trato es una  precondición  para  el  aprendizaje. (2013, p. 16). Según  Bustamante,  Terán  e  Hinojosa (2013),  datos  del  Ministerio  de  Educación muestran  esa  clara  apuesta  a  favor  de  la inclusión  a  través  de  la  progresiva disminución  de  la  exclusión  desde  el  año 2003  hasta  el  2011  y  la  inclusión  casi universal  desde  los  7  a  los  11  años.  No obstante, los autores también señalan que aún  quedan  notables  desafíos  para  lograr superar  las  profundas  inequidades  y desequilibrios  económicos,  sociales,  de género  y  culturales  que  caracterizan  a  la sociedad ecuatoriana y que se visibilizan en los promedios nacionales. Dentro  de  ese  marco  nacional,  la provincia  de  Esmeraldas  presenta  grandes necesidades  y  desafíos  en  el  terreno educativo, pues los esfuerzos que se vienen realizando  necesitan  de  más  tiempo  para reflejar  cambios  significativos  tanto  en  los resultados académicos del  alumnado  como en  la  mejora  de  la  convivencia  y  clima escolar. Se ha de considerar que se parte de niveles  de  desarrollo  social  muy bajos,  en los que gran parte de la población no tiene las  necesidades  básicas  satisfechas (Secretaría  Nacional  de  Planificación  y Desarrollo [SENPLADES], 2013, p.29). Estos desafíos se ven reflejados, por un lado, en un  porcentaje  de  analfabetismo  aún bastante significativo: 9,8% en el año 2010 para  las  personas  mayores  de  15  años (Instituto  Nacional  de  Estadística  del Ecuador  [INEC],  2010).  Por  otro  lado,  la cobertura de la educación en la provincia es de  87,7%  y  el  porcentaje  de  hogares  con niños/as  que  no  acuden  a  un  centro educativo es de 9,5%. Además, la educación básica presenta una tasa  de  escolarización del 86,1%; y existe un 14% de niños que no están  registrados  en  el  sistema  educativo (INEC, 2010).  Estos  retos  en  el  terreno  educativo  se enmarcan  en  unas  condiciones  de  vida  de las  familias  de  Esmeraldas  caracterizadas por la precariedad: en el cantón Esmeraldas, un 12,3% de la población reside en hogares sin  agua  entubada;  un  14,7%  reside  en viviendas tipo mediagua, ranchos, covachas u otras; un 8,3% de los hogares no disponen de  conexión  a  sistemas  de  eliminación  de excretas; y un 5,1% reside en viviendas sin electricidad (Secretaría Nacional de Gestión de  Riesgos,  Comisión  Europea,  PNUD, Universidad Técnica del Norte, 2013). El censo de población y vivienda (2010) sitúa a Esmeraldas entre las cinco provincias con  más  pobreza  (necesidades  básicas insatisfechas),  con  un  78,3%,  en  quinto lugar, detrás de Sucumbíos -87%-, Orellana -85%-, Los Ríos -79,5%-y Napo -78,6%-. La falta de acceso a las fuentes de empleo hace que  en  muchas familias escaseen los recursos y por tanto  muchas niñas y niños no dispongan de una buena alimentación y buenas  condiciones  de  descanso  (sueño), elementos  esenciales  para  un  buen desempeño  escolar  (Foronda,  2009).  A estos  factores  de  riesgo  se  suman  otros aspectos presentes en muchos de los barrios de la ciudad de  Esmeraldas y en particular en  el  referido  en  este  estudio  de  caso, el barrio  20  de  Noviembre,  como  son:  la desestructuración  de  los  hogares, situaciones de violencia intrafamiliar, pocas posibilidades  de  apoyo  de  la  familia  en  la realización  de  tareas,  la  exposición  a situaciones de compraventa y propuesta de consumo  de  drogas,  la  existencia  de pandillas,  contextos  de  violencia  en  las calles y poca cobertura de salud, entre otros (Foronda, 2009). También se ha de señalar que  Esmeraldas  presentó  los  peores resultados  educativos  del  país  según  las pruebas  SER  Estudiante  del  año  2008 (Instituto Nacional de Evaluación Educativa [INEVAL],  2009),  lo que  señala  una alerta en las posibilidades de desarrollo personal, empleo  y  vida  digna  de  muchos  niños, futuros jóvenes. Stuecher  y  Foronda  (2013)  resaltan también la necesidad urgente de abordar las mejoras  educativas  en  Ecuador  (y  en particular  en  los  contextos  más desfavorecidos  como  es  el  caso  de Esmeraldas)  desde  los  principios  de  la justicia social. En  relación  a  los  contextos  educativos citados, numerosas investigaciones resaltan su  importancia  a  la  hora  de  comprender cómo  se  desenvuelve el  alumnado ante  el aprendizaje y su desarrollo escolar integral (Johnson,  Rasbury  y  Siegel,  1998;  Kreuz, Casas, Aguilar y Carbó, 2009); así como la importancia  de  la  detección  temprana  de desajuste  emocional  del  alumnado  en  la escuela  (Jaureguizar,  Bernaras,  Ibabe  y Sarasa,  2012),  que  desde  la  perspectiva sistémica-familiar  tiene  en  cuenta  los diversos contextos y actores que inciden en la vida escolar de la niñez. De hecho, desde esta  perspectiva  sistémica-familiar  “es imposible resolver una conducta aislada sin tener  en  cuenta  el  comportamiento de  los demás  elementos  que  interactúan simultáneamente  en  los  contextos  en  los que el niño interacciona” (Kreuz et al., 2009, p. 48). Desde este enfoque, a pesar de que la solicitud inicial de intervención educativa con  un  estudiante  suele  plantearse  como atención individualizada, se  hace  necesario situar  dicha  intervención  en  un  contexto más  amplio  que  tiene  en  cuenta  otros sistemas o subsistemas; es por ello que se ha  de  facilitar  la  comunicación  entre  la escuela, la familia e incluso otros agentes de la  comunidad  como  un  centro  integral  de atención pedagógica y/o clubes, entre otros (Kreuz  et  al.,  2009).  En  palabras  de  los mismos autores: Y  es  aquí  donde  la  perspectiva sistémica,  la  escuela,  los  niños,  los padres, los profesores, el personal del centro, la  familia  extensa  del niño, el psicólogo,  se  amalgaman  para  hacer una  mezcolanza  maravillosa  donde cada color es imprescindible para que resalten los otros, donde cada textura define y reconoce a las demás, donde la  diferenciación  termina  provocando una sabia unión. (Kreuz et al., 2009, p. 48). Teniendo  en  cuenta  estas  premisas,  el objetivo  general  de  este  estudio  fue identificar  las  necesidades  educativas específicas del estudiante  estudiado  con  el fin de diseñar una propuesta de intervención educativa  ajustada,  realista  e  integral, desde  el  modelo  sistémico  familiar. Partiendo  de  ese  objetivo  general  se plantearon  como  objetivos  específicos:  el identificar, desde un diagnóstico integral, las dificultades  de  aprendizaje  específicas  del estudiante en las áreas de lectura y escritura y lógico-matemática; y  formular,  con  base en  los  ámbitos  de  estudio,  actividades  y acciones específicas orientadas a la mejora del  aprendizaje  y  desarrollo  personal  del estudiante. En la actualidad el alumnado que se desenvuelve en el entorno universitario, al haber superado los niveles educativos anteriores, tiene un alto nivel de autonomía y es capaz de gestionar las tareas que  implican la  formación  académica  superior.  Sin  embargo,  los  estudiantes universitarios  con discapacidad precisan de  adaptaciones  que en diversas  situaciones  puedan favorecer su  inclusión y proceso de aprendizaje, ya que, aunque logren superar los objetivos de esta etapa con estrategias personales y apoyos no institucionales, tienen derecho a contar con adaptaciones y apoyos que les permitan hacerlo en igualdad de condiciones con respecto a sus compañeros sin discapacidad que partan de la propia institución educativa. Docentes y estudiantes y personal administrativo han de cooperar en la detección de esas necesidades para favorecer la respuesta a estas y para la construcción de un entorno universitario cada vez más inclusivo. En  esa  perspectiva  de  la  mejora  continua,  en  las  universidades  existen  los  programas de  tutoría,  diseñados  con  el  objeto  de  favorecer  el  proceso  de  aprendizaje  del  alumnado.  Los principales  objetivos  de estos programas  son el acompañamiento  al  alumnado  y la detección  de posibles dificultades para  el buen desempeño en la etapa universitaria: asimilación de  contenidos, adaptación a las metodologías usadas por los docentes, o el abordaje de las necesidades educativas específicas (N.E.E.) que pueda presentar el alumnado, entre algunas de las principales necesidades académicas; aunque cabe hacer  notar  que las tutorías también tratan de fortalecer en el alumnado otras competencias como las habilidades sociales y personales u otras genéricas y transversales como la planificación o la  organización.  Es  en  ese espacio, además del  ámbito  de  las clases ordinarias, donde pueden existir mejores posibilidades para detectar las necesidades específicas del alumnado con discapacidad. En este contexto y desde un enfoque de atención a la diversidad, en este estudio de caso se han considerado los  postulados  de García (1993), quien  expone  que un estudiante puede  tener  la necesidad de que se le dote de medios especiales para acceder a la educación tales como: equipos especiales,  técnicas  especiales  de  enseñanza,  una  modificación  del  currículum  o  inclusive  una atención particular a la estructura social y al clima emocional del que participa. Otros autores como Escurra Ortiz y Molina Argudín (2000) indican que las personas con N.E.E. pueden presentar o no discapacidad y  tener  diferencias significativas  en  el aprendizaje  respecto de  sujetos  de su  misma edad y que estas diferencias pueden darse en áreas como la manera y riqueza en que se comunican; la forma y calidad de sus movimientos; la manera en que interactúan y se relacionan; así como el modo en que acceden a la información y el ritmo en el que aprenden. Por su  parte, Parrilla  Latas (1996)  sostiene que  el aula  es un  espacio de  interacciones, un contexto  académico,  social y  socializador en  el que la  vida de  estudiantes  y profesor  transcurre tanto con características académicas como personales y emocionales; y que para que se lleve a cabo el aprendizaje  es necesario  una  consideración propia  en cuanto  a  su organización,  así como  a  la consideración de reglas y rutinas. Las perspectivas teóricas mencionadas hacen referencia a grandes retos que se han de abordar, especialmente en las universidades latinoamericanas, para lograr avances significativos en favor de la inclusión en la educación superior en cumplimiento de las declaraciones y normativas internacionales y  en  el horizonte del derecho a la educación para todas las personas (García Sánchez, Real Castelao, Alcedo Rodríguez y Aguado Díaz, 2015).En la literatura científica se observa que en las dos últimas décadas las investigaciones sobre inclusión de estudiantes con discapacidad en estudios superiores presentan un gran aumento, lo cual indica un avance legal, social y actitudinal hacia la temática. Para este estudio se  han tomado como referentes tres investigaciones que han utilizado la entrevista que se aplica en este estudio (Aguado et al., 2006; García Sánchez, 2013 y Real Castelao, 2015 ), desarrollados en universidades de España y México  y, respecto a la metodología en la temática, se han revisado investigaciones que abordan estudios de caso de estudiantes con discapacidad en la universidad (Aquino Zúñiga, García Martínez, Verónica  e  Izquierdo  2012; Gago, Malasaña y Moya, 2002  y  López  Vélez,  2008);  y, en especial, un estudio  realizado  en Ecuador  en la  Universidad  Politécnica Salesiana  (Gallegos,  2011),  único referente nacional encontrado que describe el caso de una estudiante universitaria con sordoceguera. La Ponticia Universidad Católica del Ecuador Sede  Esmeraldas  (PUCESE),  institución  en la que se desarrolla este estudio, es una universidad cofinanciada, amparada en la Ley de Educación Superior del Ecuador  (2010). Al ser  cofinanciada  cuenta con el aporte  económico  del estado que permite subsidiar la educación de 1500 estudiantes aproximadamente. Como institución de educación superior ecuatoriana, sigue los  reglamentos y lineamientos que marcan el Consejo de Evaluación, Acreditación y Aseguramiento de la Calidad de la Educación Superior (CEAACES) y la Secretaría de Educación Superior, Ciencia, Tecnología e Innovación (SENESCYT). Esta última ha implementado un Programa de Becas para personas con discapacidad y a través del acuerdo 2014_029  establece que las universidades cumplirán normas de accesibilidad para personas con discapacidad y contarán con un sistema de becas para dichas personas; por lo que estudios como el presente son claves para iniciar los procesos de inclusión que marca la normativa. El presente caso de estudio evidencia una realidad poco documentada en el ámbito universitario, debido a las escasas oportunidades de realizar un seguimiento específico e integral a alumnado con N.E.E. en este nivel educativo. No obstante, es necesario profundizar en el análisis de las dificultades que enfrenta este tipo de alumnado y hacerlo desde su propia perspectiva principalmente, para así proveer a la comunidad universitaria, y específicamente al cuerpo docente, de información relevante para  la adaptación  de  su práctica  educativa  a  dichas necesidades,  buscando  las herramientas  y adaptaciones que les permitan llevar a cabo de manera más eficiente el acto educativo. En ese sentido, este estudio de caso puede ser un punto de partida importante para tener en cuenta en las universidades como referente para otros casos de alumnado con N.E.E. que existan; así como para que los docentes empiecen a adaptar metodologías, recursos y procesos de evaluación que favorezcan un mejor desempeño académico del alumnado con N.E.E. Ante la dificultad constatada en diversos estudios del ámbito de la violencia de género, de poder detectarla tempranamente, así como la no identificación por parte de adolescentes y jóvenes de los estereotipos y actitudes sexistas, Marchal, Brando, Montes y Tomás (2018) presentan un instrumento para la evaluación de actitudes machistas, violentas y estereotipos en adolescentes que puede ser de utilidad para abordar dicha problemática y orientar intervenciones educativas. Desde el marco especificado, en este estudio se presentan los resultados de la prueba piloto de aplicación de tres instrumentos de medida: uno de medición del grado de CMI v.2, la escala de valores de Schwartz (2003) y el instrumento de evaluación de actitudes machistas, violentas y estereotipos en adolescentes (EVAMVE) (Marchal et al., 2018). El propósito de ello ha sido examinar, en la muestra seleccionada, la idoneidad de cada uno de los instrumentos especificados como parte del diseño de una intervención para analizar la competencia mediática e informacional, los valores y la identificación de actitudes machistas a través de la implementación de un programa de intervención con alumnado universitario de Esmeraldas (Ecuador). No compartían la visión  de que  la orientación  Intrínseca  es  más madura,  ya que  esta forma  de religiosidad resultaba muchas veces excesivamente dogmática, rígida y acrítica. Esta  tercera orientación  correspondería mejor  a  una vivencia  más plena  de la religión  que  conlleva  actitud  crítica,  inconformismo,  capacidad  de  asumir  la complejidad, la tensión de la duda y actitud positiva de búsqueda de sentido. La otra variable que aborda esta investigación son los valores. Schwartz (2003) planteó un conjunto de valores universales tratando de encontrar una estructura coherente entre ellos. Los valores se sitúan en escala constituyendo un sistema ordenado  de  prioridades,  y  desde  dicha  escala  orientan  las  actitudes  y  las acciones. Las diferencias de valores entre personas tienen su causa en elementos genéticos  que  marcan el temperamento  individual,  los contextos sociales  y las experiencias personales. Sobre  esta  base  Ros  y  Schwartz  (1995)  elaboraron  una  teoría  transcultural compuesta  de  10  valores,  agrupados  en  4  dimensiones.  Un consenso generalizado expresado en varios estudios de la UNESCO sitúa la Declaración de Grünwald (UNESCO, 1982) como el origen de la conceptualización de la AMI puesto que en ella se proclamó la necesidad de favorecer, desde los sistemas políticos y educativos, la capacidad de comprensión de los fenómenos de comunicación por parte de la ciudadanía. Livingstone (2011) considera que existen muy altas expectativas en relación a lo que la AMI puede generar en la ciudadanía: el empoderamiento, la alfabetización critica, el compromiso democrático y la cultura de la participación en un contexto actual profundamente mediatizado; y conceptualiza la AMI como un proceso orientado al logro de la competencia de acceso, comprensión y evaluación critica de rasgos diversos tanto de los medios, como de los productos generados por estos; y también de la competencia para originar mensajes comunicativos en contextos diferentes La competencia referida comprende todos los medios (Buckingham, 2005), tiene como propósito generar conciencia en la ciudadanía sobre las múltiples formas en las que los mensajes mediáticos inundan sus vidas; y su relación con la inclusión de la ciudadanía en la actual sociedad del conocimiento; por lo que constituye una competencia fundamental para todas las personas El Parlamento Europeo (2018) la considera una competencia esencial tanto para los jóvenes como para los adultos."

#Articulo en base de datos autor4 - si esta en la base de datos
#nuevo_articulo="Lo encontrado en este tipo de ríos bajos de la costa, o evidencia no solo la llegada, sino sobre todo el ; establecimiento y la movilización de poblaciones e de peces introducidos. La plasticidad biológica, sumada a una plasticidad comportamental, permite ajustes inmediatos en la respuesta a nuevas circunstancias (Deacon, y Magurran, a 2016), lo cual es una característica de las a especies eurioicas (amplia resistencia a factores ambientales), como muchas de aquellas que son i introducidas y que terminan siendo invasoras o (Barquero-Gonzáles et al., 2018). La presencia de especies invasoras en este tipo de ríos podría -o además generar cambios ecosistémicos que a D. futuro se expresarían en forma de cambio en los flujos biogeoquímicos, en la estructura trófica, en la estructura del hábitat (García-Berthou et al., 2015) y/o en fuertes modificaciones de los ciclos de nutrientes, por ejemplo, nitrógeno o fósforo (Eby et al., 2006). Por tanto, la identificación temprana de los cambios, que la presencia de especies invasoras está generando en las especies locales, es de gran importancia para evitar que algún momento la pérdida de biodiversidad y la homogenización biótica (Olden et al., 2011) sea común en estos ambientes tropicales únicos. Más allá de cualquier discusión, es importante entender que la mejor solución a los problemas en este sentido, siempre será trabajar agresivamente para reducir la tasa de llegada de las especies que puedan lograr ser invasoras. La interpretación de la diversidad biológica puede resultar muy compleja, ya que existe una dinámica basada en el origen y en la diversificación de las especies, así como en los efectos antrópicos, tales como la introducción de especies invasoras, cuya consecuencia es muy dificil de predecir. La competencia entre especies nativas e introducidas no siempre es equitativa ya que las especies invasoras comúnmente cuentan con ventajas adaptativas que le otorgan beneficios ante las especies locales. Este trabajo analiza la variación temporal (2012 y 2017), tanto en composición como en distribución de la ictiofauna a lo largo del río Atacames, noroccidente del Ecuador. En 2012 se registró: 1) la presencia de Astyanax ruberrimus que se extinguió localmente para el 2017; y 2) la presencia de la tilapia (Orechromis niloticus) como única especie introducida a la que se sumó el guppy (Poecilia gillii) para el 2017. Se identifica también los efectos de la reciente presencia de Poecilia gillii sobre la especie nativa Pseudopoecilia fria; mediante un análisis poblacional (distribución longitudinal. parámetros de crecimiento y morfometría): los resultados muestran un desplazamiento de P. fria hacia la parte alta del río, un aceleramiento en su crecimiento y por lo tanto una madurez anticipada con la consecuente disminución del tamaño corporal y finalmente un cambio en el alto del cuerpo con deslizamiento posterior de la aleta pectoral. Finalmente, un análisis del estado de bienestar general de las especies que habitan este rio: se identifica a O. niloticus con el mejor estado y a P. fría con el más bajo. Estos resultados destacan la importancia que tiene una gestión apropiada de nuestros recursos bioacuáticos, ante el efecto de las especies introducidas y la acción del hombre; apremia seguir con estudios más detallados para entender mejor las consecuencias de este tipo de impactos.Estos resultados destacan la importancia que tiene una gestión apropiada de nuestros recursos bioacuáticos, ante el efecto de las especies introducidas y la acción del hombre; apremia seguir con estudios más detallados para entender mejor las consecuencias de este tipo de impactos La región noroccidental de la Costa Ecuatoriana incluye una diversa combinación de cuencas, grandes y pequeñas. De las primeras existe considerable información, tales como la cuenca de los ríos Santiago-Cayapas o la cuenca del Río Esmeraldas; esta última incluye ríos tributarios que recorren desde Quito, la capital del Ecuador (Barriga, 1994, 2012; Aguirre et al., 2019). Esta región tiene altos niveles de endemismo, gracias al aislamiento generado por la formación de los Andes hace millones de años (Eigenmann, 1921; Dodson y Gentry, 1978, 1991). Aunque recientemente han existido esfuerzos por obtener información sobre los peces de agua dulce en esta región (e.g. Jiménez-Prado et al., 2015; Valdiviezo-Rivera et al., 2018), se conoce muy poco sobre la mayoría. Sin embargo, se continúan descubriendo nuevas especies y existen problemas taxonómicos en algunos grupos, aún por resolver, como en los carácidos o en los astroblépidos (Tan y Armbruster, 2012; Román-Valencia et al., 2013; Lujan et al., 2015; Provezano y Barriga, 2017). Se conoce menos sobre la ecología de la mayoría de las especies y menos aún sobre el funcionamiento de las comunidades de peces en relación con sus ecosistemas acuáticos. Además, se trata de una zona que es ampliamente explotada por el ser humano, quien ha provocado transformación de hábitats para la agricultura, la minería, la construcción de ciudades muy pobladas y represas en ríos (Molinero et al., 2019). En esta región existen cuencas pequeñas, que nacen en altitudes no superiores a los 300 msnm, gracias a la presencia de montañas bajas costeras, que están aisladas y que desembocan directamente al mar, que además son perennes porque responden a patrones climáticos de abundante humedad, precipitación y aporte de aguas subterráneas (Molinero, 2019). La vegetación, en este tipo de cuencas, va desde el bosque tropical siempre verde en la cabecera y en la parte media del cauce hasta el bosque tropical semideciduo en su parte baja (MAE, 2005). Estos ríos son de bajo gradiente (1,0 – 2,3%), que forman un cauce compuesto por grava-arena y tienen una profundidad media que oscila entre 30 a 70 cm durante la estación seca (Jiménez-Prado y Vázquez, 2021). Los ríos Atacames y Súa, ubicados en la provincia de Esmeraldas, son un ejemplo de este tipo de cauces; son contiguos y presentan altitudes no mayores a los 200 msnm, tienen una extensión del cauce cercana a los 40 km y un área de drenaje con alrededor de los 300 km2 . Estas cuencas, se caracterizan por la presencia de pastos y cultivos, en un 79,5 %; zona urbana en un 3,2 % y parches de bosque natural intervenido, en un 17,3 % (Molinero, 2019). Durante el verano, que va de julio a diciembre, se presentan precipitaciones esporádicas y escasas (132 mm) y temperatura ambiental promedio de 25,2 °C (mínima = 20,0; máxima = 38,0); mientras que en el invierno las lluvias son esporádicas pero abundantes (459,6 mm) y la temperatura promedio alcanza los 27,1 °C (mínima = 21,0; máxima = 37,0). A lo largo del gradiente altitudinal, estas cuencas pueden ser subdivididas en tres zonas (alta, media y baja), según características geomorfológicas (Montaño, 2018), de orden del río (Strahler, 1957) y de altura sobre el nivel del mar (Figura 1). La zona alta del río se caracteriza por presentar mayor pendiente y altitud (colinas altas con drenaje promedio de 2,65 km2 ), lo que forma vertientes de agua con corriente moderada, aunque sus cauces son pequeños (orden de río 1 y 2); la zona media del río se caracteriza por tener menor pendiente y altitud (colinas bajas con drenaje promedio de 61,06 km2 ), lo que forma cauces de agua con corriente moderada y ancho de río moderado (orden de río 3 y 4); la zona baja del río se caracteriza por tener una pendiente y altitud leve (llanura con drenaje promedio de 111,8 km2 ), lo que forma cauces de agua con poca corriente y ancho de río moderado a grande (orden de río 4 a 6) (Jiménez-Prado y Vásquez, 2021). En mayo del 2010 se inició la construcción de dos represas en el río Atacames y una en el río Súa, las cuales fueron diseñadas originalmente para el control de inundaciones y como sistema de riego. A pesar de que su construcción no ha concluido, originaron represamientos de agua que en el río Atacames y Súa, funcionan como barreras artificiales (con un alto aproximado de tres metros) y actualmente están siendo aprovechadas como grandes “piscinas” para la recreación y el turismo local."

#Articulo en base de datos autor5 - no esta en la base de datos
nuevo_articulo="Desde el diseño social, el diseñador es capaz de establecer procesos comunitarios para crear productos y situaciones que mejoren las condiciones de vida.  En este artículo se incluyen los métodos de trabajo interdisciplinario entre diseñadores grácos, técnicos de atención al refugiado y emprendedores de Esmeraldas (Ecuador), concluyendo que la participación desde el diseño aporta signicativamente en la integración al nuevo contexto del emprendedor refugiado. La ambigüedad del perfil profesional da lugar a deficiencias en la asignación de sus funciones y propicia el desconocimiento que lo relega a espacios de intervención basados en estereotipos o relacionados con los oficios que originan la disciplina.Desde la academia, se ha tratado de establecer una definición que contenga de manera general las funciones del profesional del diseño, orientado a la comunicación visual y basado en sus competencias, describiéndose entonces como un gestor del proceso de comunicación median-te la construcción de composiciones grácas que se convierten en soluciones de comunicación.Surge entonces la interrogante: ¿Cuál es la metodología a seguir para desarrollar el contenido curricular de una carrera en cons-tante evolución? No existe una respuesta única, lo que se puede precisar son propuestas estratégicas cuyo fin sea construir un perfil profesional coherente con el cambio que per-mita ampliar el campo de acción del diseñador.Este artículo presenta la postura conceptual para el proceso de rediseño curricular de la carrera de diseño gráfico de la PUCE, correspondiente al año 2019, fundamentada en la evolución funcional del diseñador desde la construcción gráfica al diseño de comunicación.La  información  universitaria  es  un indicador del estado, alcance e impacto de las acciones  institucionales en  la  sociedad, además  se  reconoce  la  importancia  de  las operaciones  comunicativas  para  su desarrollo (De Aguilera, Faría, &  Baraybar, 2010). Las personas, actualmente reconocidas como usuarios, intervienen en la comunicación de forma dinámica y tienen la necesidad de reconocer con claridad la mayor cantidad de información posible, sabiendo que en los espacios digitales se gestiona en gran volumen, sin embargo, dicho crecimiento demanda mayor esfuerzo de percepción y en este punto es dónde el diseñador de comunicación trasciende de la creación de piezas de comunicación como función del diseño gráco para desarrollar ex-periencias de comunicación. Los mensajes actuales se desarrollan en entornos interactivos bidireccionales, enriquecidos con datos que aportan valor y profundizan su signicado. Esto no signica la desaparición del perl profesional del diseñador gráco como comu-nicador visual, más bien obedece al cambio para exibilizar al nuevo diseñador de comu-nicación como un profesional con capacidad de adaptación para resolver problemas que trascienden de lo visual hacia una comunicación compleja, dinámica y clara.En síntesis, los aspectos que debería contener el diseño curricular para la construcción eciente y de amplio alcance en la formación a nivel superior son:- De acuerdo con el esquema de tabajo-aprendizaje dentro de proceso formativo, la capa-cidad de relacionar los logros esperados independientemente del tiempo.- Modificar el esquema tradicional de construcción curricular a través de la selección de campos del conocimiento coherentes con la realidad y el contexto de intervención para resolver problemas.- Diseñar el currículo desde el pensamiento complejo, reconociendo la importancia de la competencia laboral como eje transformador de los esquemas educativos.- Considerar las necesidades y tendencias tanto locales como globales como parte impor-tante dentro del proceso de rediseño curricular a nivel superior.- Ante todo, aplicar un modelo curricular integrador que posibilite la construcción si-nérgica del contenido y aporte a la gestión  eciente del tiempo destinado a la formación, además de facilitar la integración de otros componentes que contribuyan a los contenidos académicos más allá de los cambios sociales, culturales o tecnológicos. La investigación se expone desde un enfoque exploratorio mixto, por tratarse de un fenómeno poco estudiado a nivel global, con nulo antecedente nacional y local, de modo que constituye un punto de partida para nuevas iniciativas. Encierra un carácter ex post-facto, donde “los hechos […] ya ocurrieron” (Hernández, Fernández & Baptista, 2014, p.198), ya que los datos pertenecen a procesos de seguimiento durante el año 2016. Desde el  punto  de vista  del diseño  de investigación, el  proyecto se  aborda  desde  un  carácter  no experimental, priorizando el  trabajo de campo y el  proceso  experiencial. No aplica, en este caso, objeto de estudio, sino obtención  de  datos a  partir de  las valoraciones de  la población. Por otro  lado,  el carácter  mixto se  da  al recoger en primer lugar la relación de  variables (ítems  Reacción, Aprendizaje, Transferencia e Impacto) de manera cualitativa  sobre las  jornadas  de capacitación  en identidad  visual  para micro-emprendimientos,  y por último una fase de análisis cuantitativo a partir de la encuesta que mide la variable Satisfacción general de la persona emprendedora. En todo caso, estas variables se reeren a la evaluación de impacto para las personas, teniendo como posibles vías futuras la estimación de impacto en la sociedad y en las empresas o emprendimientos a nivel económico5.La  muestra  de estudio  corresponde a  un  total de  26 personas,  pertenecientes  al  colectivo de  refugiados del  proyecto  ACNUR,  Género y  Emprendimiento  de  la  PUCESE,  asentadas  en la  provincia  ecuatoriana de Esmeraldas. Esta  cifra proviene del  número de  personas  refugiadas que  participaron en  el proceso de desarrollo  de aplicación  de  soluciones grácas  a  sus  micro-empresas.  A diferencia  de  los  “desplazados internos”, que  se describe  como  el grupo  de  personas  obligadas a  abandonar  su  lugar de  residencia sin cruzar fronteras por  acciones  que vulneran  sus  derechos humanos  (Mendia, s.f.), la persona  refugiada se precisa como el individuo que es perseguido por pertenecer a un determinado grupo social, etnia, religión, nacionalidad o postura política, siendo movilizadas de manera forzosa fuera de su país de origen (ACNUR, 2016). Se caracterizan por ser personas movilizadas desde Colombia por problemas derivados de la violencia6, de mediana edad, que llegan a la localidad de Esmeraldas sin documentación ni recursos y por eso recurren a la ayuda de asociaciones como ACNUR. Generalmente, sus emprendimientos tienen un nivel de avance y estabilidad bajo, rozando en algunos casos el empleo informal. Estos micro-negocios suponen la subsistencia en el nuevo contexto, del que depende el propio emprendedor y sus familiares. El rol que juega la población refugiada dentro del proceso de diseño y comunicación de sus emprendimientos es fundamental porque es quién mejor conoce los atributos y características a ser difundidos, por lo tanto, su implicación como gestor del mensaje es imprescindible para un proceso de integración comunitario. Los datos del estudio se han obtenido a partir de herramientas que sirvieron para evaluar las sesiones y procesos para  poder  llevar  la disciplina  del diseño  gráco al  campo de  la integración  social.  En concreto, las herramientas de investigación-evaluación han sido la observación participante, como técnica para llevar procedimientos conjuntos entre el colectivo refugiado junto  a los diseñadores, y  el cuestionario, aplicado  a personas refugiadas implicadas en el proyecto. La observación participante se emplea como instrumento en el trabajo de campo, rearmando la necesidad de  involucrarse en los  procesos  de  un modo directo con la muestra y entendiendo esta herramienta “no como una técnica sino como un modo participativo de detectar contextos”.La finalidad del taller fue explicar el proceso para lograr la actualización de contenidos a través de la adaptación de tópicos más relevantes de acuerdo con criterio de los intervi-nientes. Es así que previa discusión exhaustiva de los temas presentados se procede a de-sarrollar la síntesis de las mallas para las ingenierías y licenciaturas, logrando la reducción a ocho semestres y su correspondiente aprobación por parte del equipo de la Dirección de Grado de la PUCE. En esta fase se utiliza como estrategia de trabajo a la exposición magistral con la intención de socializar al grupo de trabajo los cambios enmarcados en las exigencias del Reglamento de Régimen Académico, respecto a los rediseños de carreras, explicándose a detalle las modicaciones así como las posibles opciones de mejora apli-cables para cada caso. Entre otras actividades del taller, el equipo de trabajo presentó diferentes alternativas para el ajuste de la malla. Se abordaron temas relacionados con los itinerarios, la cátedra de inglés, las asignaturas con objetivos comunes, los estudios avanzados, el aprendizaje prác-tico experimental, la vinculación, las prácticas y la unidad de titulación. Cabe mencionar que todos estos aspectos son la esencia de modelo formativo adoptado por la PUCE según las regulaciones gubernamentales, porque dotan a los estudiantes de las competencias ne-cesarias para el óptimo ejercicio profesional sin olvidar la parte social, para proyectarse como una institución de educación superior que forma profesionales integrales porque van más allá de los conocimientos técnicos para convertirse en gestores de solución y por ende su intervención aporta de manera real y productiva a la comunidad.Adicionalmente, se logró establecer que para cualquier modicación de la estructura cu-rricular diseñada en consenso por el equipo de trabajo, es un requisito fundamental el análisis de componentes como: los créditos o contenidos, eliminación o unicación de asignaturas, objetivos, costos, infraestructura, perl de salida del estudiante, considerando que de éstos dependen los resultados de aprendizaje que se propone alcanzar. Por otro lado, se desarrolló de manera conjunta el formato de presentación para los nuevos pro-yectos al Consejo de Educación Superior CES, que mostraba modicaciones en forma y contenido. La técnica aplicada para el análisis de la malla vigente fue las Mesas de Trabajo (MT), esta técnica consiste en la integración de equipos con funciones especicadas para crear de manera eciente y estructurada el proceso de rediseño. Las MT estaban integradas por representantes de la matriz Quito y las sedes de Ibarra y Esmeraldas que ofertan la carrera de diseño gráco. La actividad consistió en el estudio de contenidos junto al números de créditos corres-pondientes a cada asignatura y mediante consenso se estableció un primer modelo para la propuesta de malla rediseñada. Para lograr la síntesis de contenidos en el rediseño de la carrera de Diseño Gráco en-marcada en un criterio técnico y evitando el sesgo por parte de cualquiera de los intervi-nientes, se utilizó la estrategia de eliminar, fusionar y/o cambiar asignaturas, es decir que, las asignaturas de objetivo común se eliminaron mediante la integración entre cátedras comunes. En esta mesa de trabajo se optó por analizar cada asignatura con su respectivo contenido con el propósito de evitar la duplicación de contenidos y optimizar los recursos ajustando los créditos a 720 horas por nivel. A continuación, se explican los cambios predominantes en el ajuste de la malla. En esta fase cada sede debía trabajar en los ajustes de acuerdo con el nuevo formato de pre-sentación de proyectos según los lineamientos de Consejo de Educación Superior, CES, priorizando la revisión de los resultados de aprendizaje, contenidos de materias, pertinen-cia, perl docente, objetivos educativos, costos, datos administrativos y de infraestructura. En el caso en la PUCE como matriz, continúa el análisis de la malla curricular desde la Dirección de Grados. Como sede Esmeraldas se reunió al equipo de trabajo (currículo, investigación, prácticas y vinculación), para socializar los primeros ajustes realizados en la reunión entre sedes de la malla. Se analizó a las asignaturas y contenidos de la malla anterior en relación con el Rediseño y se explicó cada uno de los cambios y qué aspectos se consideraron para el mismo. La comunicación fue uida entre los coordinadores o responsables del rediseño de cada sede con la matriz (Quito), los cambios ejecutados eran socializados y ejecutados posterior a la aprobación conjunta. La Dirección Académica en conjunto con la Dirección de Grados proceden a socializar los formatos y plazos para la elaboración del Plan de Transición. Para la implementación de este Plan se resuelve comparar los contenidos de la carrera vigente con la malla curricular rediseñada, contemplando: asignaturas, horas semanales y horas totales. Para el cumpli-miento integral y eciente del plan se incluyen al contenido las asignaturas eliminadas, las asignaturas incorporadas y articuladas en la carrera rediseñada, relacionada con cada nivel. Dicha estrategia también aplica a las acciones especícas por nivel.Las Acciones Especícas tomadas en consideración de acuerdo con la particularidad de cada nivel para ejecutar el Plan de Transición en el año 2020 – 1 son:Caso Segundo NivelSi los estudiantes aprueban todo segundo nivel tomarían todas las asignaturas de tercer nivel, en este nivel no se ha dado ninguna modicación ni en asignaturas o créditos. Si algún estudiante reprueba cualquier asignatura que tenga prerrequisito deberá tomarla como curso autonanciado en el periodo no académico. En caso de reprobar asignaturas comunes puede tomar dos caminos."

#Articulo en base de datos autor6 - si esta en la base de datos
#nuevo_articulo=""

#Articulo en base de datos autor7 - si esta en la base de datos
#nuevo_articulo=""

#Articulo en base de datos autor8 - si esta en la base de datos
#nuevo_articulo=""

#Articulo en base de datos autor9 - si esta en la base de datos
#nuevo_articulo=""

#Articulo en base de datos autor10 - si esta en la base de datos
#nuevo_articulo=""

#Articulo nuevo desconocido
#nuevo_articulo="El uso de computadores como herramienta de aprendizaje en la educación fue propuesto por Seymour Papert en 1980, convirtiendo el aprender-pensar en construir-programar, a esto lo denominó Construccionismo. El Instituto de Tecnología de Massachusetts, en 1998 construyó robots educativos LEGO MINDSTORMS como recurso de aprendizaje, con ello se logró vincular a jóvenes con la tecnología, esta herramienta sirvió para evaluar la trasferencia de conocimiento y trabajo en equipos de estudiantes de educación básica utilizando rúbricas para la obtención de datos. El acuerdo ministerial 041-14, propone introducir en la malla curricular de Educación General Básica (EGB), tres horas de clases destinadas a clubes orientadas al campo de acción científico, interacción social y vida práctica, este acuerdo sirvió de base en la presente investigación. La presente investigación desarrollada bajo el título “Dispositivo de monitoreo basado en el internet de las cosas que obtenga evidencias de lo que acontece en el barrio 20 de noviembre de la ciudad de esmeraldas y genere una alarma de alerta en el mismo”, propone desarrollar una solución tecnológica que evidencie hechos delincuenciales en el sector utilizando dispositivos electrónicos, debido a los diferentes acontecimientos de vandalismo que atentan a la integridad de los moradores, por lo cual fue necesario utilizar un microprocesador acorde a las necesidades de la investigación, así como, un medio web para la comunicación entre los usuarios del dispositivo. Esmeraldas en las últimas décadas ha sufrido un aumento de la delincuencia que mantiene preocupada a su población; cada año el aumento o disminución de la delincuencia varía según aplicativos que se tomen en cuenta para mantener segura la población, pero a pesar de ello la delincuencia aún permanece en la ciudad. El Ministerio del Interior (2017) expone que el crimen organizado, homicidios intencionales, robo a domicilio, personas, unidades económicas, carros, motos, tráfico y microtráfico de drogas son los acontecimientos delictivos más notorios en la ciudad de Esmeraldas a cualquier hora del día. La falta de empleo, explotación del empleado no profesional y pagos irrisorios por el desempeño laboral son puntos que han favorecido al aumento de la delincuencia como tal. Un ejemplo estadístico de hace 10 años donde Diario el Universo (2008) publicó que la Policía Nacional desde el 2008 comentó que la ciudad había aumentado el nivel de delincuencia en un 15% tras el asesinato de un joven policía en la calle Malecón, una cifra muy alarmante para la pequeña población que mantiene en incertidumbre a más de un ciudadano. Años más tarde tras varias intervenciones por parte de la policía nacional ese porcentaje que en un principio iba en aumento se vuelve menos alarmante para la zona urbana del cantón, a continuación, en la figura 1 se muestran datos estadísticos que proporciona el Ministerio del Interior (2017) a los ciudadanos para analizar la variabilidad de la delincuencia año tras año. La figura 1 representa el 31.18% de disminución de robos a domicilio entre el año 2017 y el año 2016, siendo así que se suscitaron 417 robos a domicilio en 2016 y 287 robos a domicilio en 2017, en donde los días con mayor recurrencia para delinquir serían miércoles, viernes y domingos en horas de la madrugada, como se muestra en la figura 2. Por otra parte, fomentar el deporte, los distintos tipos de arte, el descubrir talentos desde la niñez no es algo a lo que se le dé mucha importancia en la ciudad, iniciativa que debería ser tomada por las autoridades competentes del asunto social en Esmeraldas. Un dato curioso es que la provincia es cuna de deportistas (como coloquialmente se le llama) que participan en diferentes disciplinas deportivas en el Ecuador y aún con estos antecedentes no se realizan los procesos adecuados para involucrar a los jóvenes y adolescentes de una población mayormente caracterizada por su alto rendimiento físico en los diferentes programas atléticos que pueden existir. En la figura 3 se pueden observar varias acciones que se pueden tomar para disminuir la delincuencia o inseguridad en la ciudad, encuesta publicada por CEDATOS basado en la investigación por parte de la ESPOL. La falta de Dios, sus principios, mandamientos, sacramentos hacen que mayormente la juventud se envuelva en círculo vicioso de perversidad y desadaptación total de la sociedad, trayendo como consecuencia un déficit de valores, moral, integridad y por su puesto falta respeto y temor ante Dios, donde atentar contra el prójimo es un acto común. El barrio “20 de Noviembre” no es la excepción en el tema, ya que también se suscitan hechos de tipo delincuencial originados mayormente por dos grupos contrincantes de jóvenes y adolescentes de acuerdo a versiones de los moradores del sector. La mayoría de involucrados en estos grupos de jóvenes y adolescentes no asisten a un establecimiento educativo lo que origina que tengan tiempo de ocio y dedicación a actividades delincuenciales, lastimando su integridad y la de la comunidad en general. El barrio carece de zonas de recreación, de programas o actividades que incluyan a estos jóvenes y adolescentes, ya que la falta de estas actividades le da espacio a que los vicios se adueñen de estos jóvenes. Lanzamientos de objetos peligrosos, detonaciones de armas de fuego, robos, atracos a transeúntes y conductores son varios de los acontecimientos que se suscitan en el barrio perjudicando una vez más la integridad de los moradores y la de sus propiedades privadas. Todo este ambiente trae preocupación y temor a los moradores del barrio, ya que no existen cámaras de seguridad o un dispositivo tecnológico que puedan captar todas estas anomalías y presentarlas como evidencia de actos de índole delincuencial ante la Policía Nacional del Ecuador para colocar la respectiva denuncia. La incorporación de las TIC en los procesos educativos, en la actualidad, es indiscutible y son varias las herramientas tecnológicas desarrolladas con el fin de apoyarlos, sin embargo, ninguna herramienta tecnológica, por sí sola, puede aportar a la educación sin estar ligada a una estrategia didáctica apropiada. Para investigadores como Oyola y Gómez (2012), Ferro y Martínez (2009), Castillo (2008), Mora (2003), entre otros, el proceso de enseñanza aprendizaje se ve fortalecido con el empleo de estrategias didácticas que garantizan el uso apropiado de las tecnologías como recursos educativos. Dentro de la dinámica pedagógica, el docente, desde una postura constructivista, debe promover procesos educativos que permitan la participación de los estudiantes a través de actividades previamente planificadas y diseñadas para propiciar su desarrollo intelectual (Castro, Peley y Morillo, 2006). En este sentido, es primordial que el docente considere las TIC como un recurso de apoyo que debe insertar al delinear su estrategia didáctica. Para Ocampo (2003), las estrategias didácticas son procesos o secuencias de actividades que sirven de base a la realización de tareas intelectuales y que se eligen con el propósito de facilitar la construcción, permanencia y transferencia de conocimientos (p. 9). Por otro lado, Carrasco (2004), define una estrategia didáctica de enseñanza como el conjunto de instrucciones que el docente diseña y aplica para beneficiar los procesos de enseñanza-aprendizaje Bajo este contexto, según Díaz y Hernández (2012), las estrategias didácticas de enseñanza pueden ser: pre-instruccionales, co-instruccionales y post-instruccionales; las que están respectivamente ligadas a diferentes momentos de una clase: inicio, en el que se plantea lo que se va a aprender y cómo se lo hará; desarrollo, en el que se exponen los contenidos y se aplican técnicas motivacionales para mantener el interés de los estudiantes; cierre, en el que se comprueba si los objetivos propuestos al inicio de la clase se han alcanzado. En el campo de las matemáticas, la enseñanza-aprendizaje debe estar en concordancia con el avance tecnológico, para lo cual, debe estar ligada a una metodología que favorezca el desarrollo cognitivo de los estudiantes, que estimule su pensamiento crítico y su autorregulación, que tienda el puente entre la educación tradicional y aquella que emplea la tecnología a su favor (García, Escalera y Navarro, 2011). En el entorno educativo, varios son los recursos tecnológicos que se utilizan, entre ellos: software educativo, simuladores, herramientas multimedia y de presentación, correo electrónico, blogs, etc., algunos de ellos creados específicamente como recursos didácticos y otros que se han adaptado. En concordancia a lo anteriormente expuesto, Herrera, Novelo, Díaz y Hernández (2016), opinan que dentro de las TIC, los software educativos especializados también son herramientas que se pueden tomar como estrategias para la enseñanza-aprendizaje de las matemáticas (p. 8). De igual manera, Morales y Peña (2013) argumentan que el empleo de herramientas tecnológicas educativas para la enseñanza de matemáticas puede ser fructífero si están acompañadas de estrategias apropiadas. En razón de lo mencionado, el uso de cualquiera de estos recursos no es una garantía de que el proceso educativo se lleve adecuadamente y que su efectividad se vea reflejada en los logros alcanzados por los estudiantes; de ahí que es necesario que el docente planifique su clase y establezca la herramienta tecnológica que, de acuerdo con sus características, se adapte mejor a su estrategia didáctica. Para este proyecto se considera como unidad de estudio el capítulo de lógica matemática, abordando los siguientes temas: proposiciones, conectores lógicos, lectura y escritura lógicas contemplada en el syllabus de Matemática de primer nivel de la Escuela de Sistemas de la PUCE Esmeraldas. Con esta referencia se diseña la estrategia didáctica con un enfoque constructivista, considerando los tres momentos de una clase: inicio, desarrollo y cierre; y contemplando una evaluación tanto formativa como sumativa. En base a los contenidos y actividades a realizar, se considera pertinente emplear como recurso digital de apoyo un software educativo, que permita la creación y manejo de contenidos, que además resulte ser amigable para el usuario; de acceso libre y gratuito. En función de estos requerimientos y tras un análisis previo, se considera que la herramienta tecnológica idónea para apoyar la estrategia es el software educativo eXe-Learning. Una vez estructurada la estrategia didáctica apoyada con el recurso digital eXe-Learning es importante determinar si su aplicación produce algún efecto en el proceso de enseñanza-aprendizaje de los estudiantes, para ello se plantea la siguiente hipótesis: La media de las calificaciones del test aplicado a los estudiantes, que recibieron las clases con la estrategia apoyada con eXe-Learning es mayor que la media de las calificaciones de aquellos que recibieron las clases sin la estrategia apoyada con eXe-Learning. A continuación, se detalla la metodología empleada, el diseño de la estrategia apoyada con el eXe-Learning, los resultados obtenidos al aplicarla y finalmente se exponen las conclusiones generadas por esta investigación."


In [66]:
# Preprocesamiento del nuevo artículo
nuevo_articulo_limpio = preprocesar_texto(nuevo_articulo)

In [67]:
# Crear perfil del nuevo artículo
nuevo_articulo_tfidf = tfidf_vectorizer.transform([nuevo_articulo_limpio])
nuevo_articulo_w2v = obtener_vector_promedio(nuevo_articulo_limpio, model_w2v)
nuevo_articulo_combined = np.hstack((nuevo_articulo_tfidf.toarray(), nuevo_articulo_w2v.reshape(1, -1)))
# Escalar características al rango [0, 1]
nuevo_articulo_scaled = scaler.transform(nuevo_articulo_combined.reshape(1, -1))

In [68]:
# Crear la tabla para almacenar los resultados
resultados_nuevo_articulo = []
    
# Realizar predicción para el nuevo artículo
for nombre, modelo in modelos.items():
    autor_predicho = modelo.predict(nuevo_articulo_scaled)
    probabilidad_prediccion = np.max(modelo.predict_proba(nuevo_articulo_scaled))
    autor_predicho_decodificado = label_encoder.inverse_transform([autor_predicho])[0]
    resultados_nuevo_articulo.append([nombre, autor_predicho_decodificado, probabilidad_prediccion])



C:\Users\User\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_

In [69]:
# Mostrar resultados en una tabla
print("\nNuevo Artículo:")
print(tabulate(resultados_nuevo_articulo, headers=['Modelo', 'Autor Predicho', 'Probabilidad'], tablefmt='grid'))


Nuevo Artículo:
+---------------------+------------------+----------------+
| Modelo              | Autor Predicho   |   Probabilidad |
+=====================+==================+================+
| SVC Lineal          | autor5           |       0.890816 |
+---------------------+------------------+----------------+
| SVC RBF             | autor2           |       0.195202 |
+---------------------+------------------+----------------+
| Random Forest       | autor5           |       0.17     |
+---------------------+------------------+----------------+
| Regresion Logistica | autor5           |       0.821547 |
+---------------------+------------------+----------------+
